# Capstone Project: Create a Customer Segmentation Report for Arvato Financial Services

In this project, you will analyze demographics data for customers of a mail-order sales company in Germany, comparing it against demographics information for the general population. You'll use unsupervised learning techniques to perform customer segmentation, identifying the parts of the population that best describe the core customer base of the company. Then, you'll apply what you've learned on a third dataset with demographics information for targets of a marketing campaign for the company, and use a model to predict which individuals are most likely to convert into becoming customers for the company. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

The versions of those two datasets used in this project will include many more features and has not been pre-cleaned. You are also free to choose whatever approach you'd like to analyzing the data rather than follow pre-determined steps. In your work on this project, make sure that you carefully document your steps and decisions, since your main deliverable for this project will be a blog post reporting your findings.

In [1]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# magic word for producing visualizations in notebook
%matplotlib inline

## Part 0: Get to Know the Data

There are four data files associated with this project:

- `Udacity_AZDIAS_052018.csv`: Demographics data for the general population of Germany; 891 211 persons (rows) x 366 features (columns).
- `Udacity_CUSTOMERS_052018.csv`: Demographics data for customers of a mail-order company; 191 652 persons (rows) x 369 features (columns).
- `Udacity_MAILOUT_052018_TRAIN.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 982 persons (rows) x 367 (columns).
- `Udacity_MAILOUT_052018_TEST.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 833 persons (rows) x 366 (columns).

Each row of the demographics files represents a single person, but also includes information outside of individuals, including information about their household, building, and neighborhood. Use the information from the first two files to figure out how customers ("CUSTOMERS") are similar to or differ from the general population at large ("AZDIAS"), then use your analysis to make predictions on the other two files ("MAILOUT"), predicting which recipients are most likely to become a customer for the mail-order company.

The "CUSTOMERS" file contains three extra columns ('CUSTOMER_GROUP', 'ONLINE_PURCHASE', and 'PRODUCT_GROUP'), which provide broad information about the customers depicted in the file. The original "MAILOUT" file included one additional column, "RESPONSE", which indicated whether or not each recipient became a customer of the company. For the "TRAIN" subset, this column has been retained, but in the "TEST" subset it has been removed; it is against that withheld column that your final predictions will be assessed in the Kaggle competition.

Otherwise, all of the remaining columns are the same between the three data files. For more information about the columns depicted in the files, you can refer to two Excel spreadsheets provided in the workspace. [One of them](./DIAS Information Levels - Attributes 2017.xlsx) is a top-level list of attributes and descriptions, organized by informational category. [The other](./DIAS Attributes - Values 2017.xlsx) is a detailed mapping of data values for each feature in alphabetical order.

In the below cell, we've provided some initial code to load in the first two datasets. Note for all of the `.csv` data files in this project that they're semicolon (`;`) delimited, so an additional argument in the [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) call has been included to read in the data properly. Also, considering the size of the datasets, it may take some time for them to load completely.

You'll notice when the data is loaded in that a warning message will immediately pop up. Before you really start digging into the modeling and analysis, you're going to need to perform some cleaning. Take some time to browse the structure of the data and look over the informational spreadsheets to understand the data values. Make some decisions on which features to keep, which features to drop, and if any revisions need to be made on data formats. It'll be a good idea to create a function with pre-processing steps, since you'll need to clean all of the datasets before you work with them.

### Load the data and basic evaluation
- read in csv files `Udacity_AZDIAS_052018.csv` and `Udacity_CUSTOMERS_052018.csv`
- basic data evaluation: shape, head, info, describe, isnull

In [2]:
# load in the data
azdias = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_AZDIAS_052018.csv', sep=';')
customers = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_CUSTOMERS_052018.csv', sep=';')

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# basic data evaluation AZDIAS
print(azdias.shape)
print('----------------------------------------------------------------------------------------------------------------------')
print(azdias.head())
print('----------------------------------------------------------------------------------------------------------------------')
print(azdias.info())
print('----------------------------------------------------------------------------------------------------------------------')
print(azdias.describe())
print('----------------------------------------------------------------------------------------------------------------------')
print(azdias.isnull().sum())

(891221, 366)
----------------------------------------------------------------------------------------------------------------------
      LNR  AGER_TYP  AKT_DAT_KL  ALTER_HH  ALTER_KIND1  ALTER_KIND2  \
0  910215        -1         NaN       NaN          NaN          NaN   
1  910220        -1         9.0       0.0          NaN          NaN   
2  910225        -1         9.0      17.0          NaN          NaN   
3  910226         2         1.0      13.0          NaN          NaN   
4  910241        -1         1.0      20.0          NaN          NaN   

   ALTER_KIND3  ALTER_KIND4  ALTERSKATEGORIE_FEIN  ANZ_HAUSHALTE_AKTIV  \
0          NaN          NaN                   NaN                  NaN   
1          NaN          NaN                  21.0                 11.0   
2          NaN          NaN                  17.0                 10.0   
3          NaN          NaN                  13.0                  1.0   
4          NaN          NaN                  14.0                  3.0

In [4]:
# basic data evaluation CUSTOMERS
print(customers.shape)
print('----------------------------------------------------------------------------------------------------------------------')
print(customers.head())
print('----------------------------------------------------------------------------------------------------------------------')
print(customers.info())
print('----------------------------------------------------------------------------------------------------------------------')
print(customers.describe())
print('----------------------------------------------------------------------------------------------------------------------')
print(customers.isnull().sum())

(191652, 369)
----------------------------------------------------------------------------------------------------------------------
      LNR  AGER_TYP  AKT_DAT_KL  ALTER_HH  ALTER_KIND1  ALTER_KIND2  \
0    9626         2         1.0      10.0          NaN          NaN   
1    9628        -1         9.0      11.0          NaN          NaN   
2  143872        -1         1.0       6.0          NaN          NaN   
3  143873         1         1.0       8.0          NaN          NaN   
4  143874        -1         1.0      20.0          NaN          NaN   

   ALTER_KIND3  ALTER_KIND4  ALTERSKATEGORIE_FEIN  ANZ_HAUSHALTE_AKTIV  \
0          NaN          NaN                  10.0                  1.0   
1          NaN          NaN                   NaN                  NaN   
2          NaN          NaN                   0.0                  1.0   
3          NaN          NaN                   8.0                  0.0   
4          NaN          NaN                  14.0                  7.0

### Data Preprocessing

- Explore features (Differences in columns)

- Check data types

- Explore features using `DIAS Attributes - Values 2017.xlsx`

- Missing Values 
    - Convert Unknown to NaN
    - Columns with NaN (distribution of % missing --> decide what to drop)
    - Rows with NaN (distribution of % missing --> decide what to drop)

- Datatypes: Classify attribute types (https://www.mygreatlearning.com/blog/types-of-data/)
    - Quantitative Attributes (Numerical)
        - Discrete / Continous data
    - Qualitative Attributes (Categorical)
        - Nominal / Ordinal
    - Impute NaN for each data type?

- Encoding / Standardization (OneHotEncoder / StandardScaler)

#### Explore Features 

In [5]:
# differences in columns
azdias_cols = set(azdias.columns)
customers_cols = set(customers.columns)
customers_cols - azdias_cols


{'CUSTOMER_GROUP', 'ONLINE_PURCHASE', 'PRODUCT_GROUP'}

There are 3 columns in the customers data set not included in azdias data set: 'CUSTOMER_GROUP', 'ONLINE_PURCHASE', 'PRODUCT_GROUP'.

#### Check Data Types

In [6]:
# check data types azdias
azdias.get_dtype_counts()

float64    267
int64       93
object       6
dtype: int64

In [7]:
# check data types customers
customers.get_dtype_counts()

float64    267
int64       94
object       8
dtype: int64

In [8]:
print('Categorical features in AZDIAS:\n', azdias.select_dtypes(['object']).head(5))
print('----------------------------------------------------------------------------------------------------------------------')
print('Categorical features in CUSTOMERS:\n', customers.select_dtypes(['object']).head(5))

Categorical features in AZDIAS:
   CAMEO_DEU_2015 CAMEO_DEUG_2015 CAMEO_INTL_2015 D19_LETZTER_KAUF_BRANCHE  \
0            NaN             NaN             NaN                      NaN   
1             8A               8              51                      NaN   
2             4C               4              24            D19_UNBEKANNT   
3             2A               2              12            D19_UNBEKANNT   
4             6B               6              43               D19_SCHUHE   

         EINGEFUEGT_AM OST_WEST_KZ  
0                  NaN         NaN  
1  1992-02-10 00:00:00           W  
2  1992-02-12 00:00:00           W  
3  1997-04-21 00:00:00           W  
4  1992-02-12 00:00:00           W  
----------------------------------------------------------------------------------------------------------------------
Categorical features in CUSTOMERS:
   CAMEO_DEU_2015 CAMEO_DEUG_2015 CAMEO_INTL_2015 D19_LETZTER_KAUF_BRANCHE  \
0             1A               1              13  

There are 6 categorical features in the azdias data set but 8 categorical featrues in the customers data set. 

The feature information to these categorical features are the following:
- CAMEO_DEU_2015 / CAMEO_DEUG_2015 / CAMEO_INTL_2015: These describe the sociodemographic background of a person or household based on the CAMEO classification system.
- D19_LETZTER_KAUF_BRANCHE: Industry in which last purchase was made
- EINGEFUEGT_AM: Date where record was added 
- OST_WEST_KZ: Inidcates whether a house is located in West Germany / Federal Republik of Germany / FRG (W) or East Germany / German Democratic Republik / GDR (O)
- PRODUCT_GROUP: Product Group where the transactional activity has taken place
- CUSTOMER_GROUP: Classification of the customer as MULTI_BUYER or SINGLE_BUYER 

#### Explore Features using DIAS Attributes - Values 2017.xlsx

In [9]:
# explore features using DIAS Attributes - Values 2017.xlsx
attr_values = pd.read_excel('DIAS Attributes - Values 2017.xlsx', usecols='B:E', dtype='str')
attr_values.head()

,Attribute,Description,Value,Meaning
0,AGER_TYP,best-ager typology,-1,unknown
1,nan,nan,0,no classification possible
2,nan,nan,1,passive elderly
3,nan,nan,2,cultural elderly
4,nan,nan,3,experience-driven elderly


There are several values in the features that have the meaning "unknown". These should be replaced by NaN in the dataset in order to perform a correct estimation and cleanup of missing values.
Among numerical characteristics, there are many whose values have natural, ordered categories. These need to be identified and classified as ordinal or categorical data.

In [10]:
# identify values with meaning unknown
unknown_values = attr_values[attr_values["Meaning"].isin(["unknown","unknown / no main age detectable"])]
unknown_values.head()

,Attribute,Description,Value,Meaning
0,AGER_TYP,best-ager typology,-1,unknown
5,ALTERSKATEGORIE_GROB,age classification through prename analysis,"-1, 0",unknown
11,ALTER_HH,main age within the household,0,unknown / no main age detectable
33,ANREDE_KZ,gender,"-1, 0",unknown
40,BALLRAUM,distance to next urban centre,-1,unknown


#### Missing Values
- Convert Unknown to NaN
- Columns with NaN (distribution of % missing --> decide what to drop)
- Rows with NaN (distribution of % missing --> decide what to drop)

In [11]:
# Create dict of missing attribute-value combinations
unknown_values_dict = {}
for _, row in unknown_values.iterrows():
    key = row["Attribute"]
    unknown_values_dict[key] = row["Value"].split(", ")
    

In [12]:
# Manually add missing codes from other/similar attributes 
unknown_values_dict["CAMEO_INTL_2015"] = ['XX']
unknown_values_dict["CAMEO_DEUG_2015"] = ['X','XX']
unknown_values_dict["CAMEO_DEU_2015"] =['XX']
unknown_values_dict["GEBURTSJAHR"] = ['0']


In [13]:
def convert_unknown_to_nan(df, keys_dict):
    """
    Replaces given keys from keys_dict to np.nan in df
    
    Input:
        df (DataFrame): Dataset for which keys to need to be converted nan 
        keys_dict: dictionary of column names with keys that need to be converted to nan

    """
    # Search for unknown keys in each column and replace

    for column_name in keys_dict:
        if column_name in df.columns:       
            keys_values = keys_dict[column_name]
            for key in keys_values:
                if key == 'X' or key == 'XX':
                    key = str(key)
                else:
                    key = int(key)
                
                df[column_name].replace(key, np.NaN, inplace=True)

In [14]:
# Number of missing values before converted to nan
azdias.isnull().sum().sort_values(ascending=False)

ALTER_KIND4                    890016
ALTER_KIND3                    885051
ALTER_KIND2                    861722
ALTER_KIND1                    810163
EXTSEL992                      654153
KK_KUNDENTYP                   584612
ALTERSKATEGORIE_FEIN           262947
D19_LETZTER_KAUF_BRANCHE       257113
D19_LOTTO                      257113
D19_VERSI_ONLINE_QUOTE_12      257113
D19_BANKEN_ONLINE_QUOTE_12     257113
D19_SOZIALES                   257113
D19_GESAMT_ONLINE_QUOTE_12     257113
D19_KONSUMTYP                  257113
D19_TELKO_ONLINE_QUOTE_12      257113
D19_VERSAND_ONLINE_QUOTE_12    257113
KBA05_MOTOR                    133324
KBA05_MOD8                     133324
KBA05_MOD4                     133324
KBA05_MOD3                     133324
KBA05_MOD2                     133324
KBA05_SEG1                     133324
KBA05_MOD1                     133324
KBA05_MAXVORB                  133324
KBA05_MAXSEG                   133324
KBA05_MAXHERST                 133324
KBA05_MAXBJ 

In [15]:
# Convert unknowns to nan
convert_unknown_to_nan(azdias, unknown_values_dict)

In [16]:
# Number of missing values AFTER converted to nan
#azdias.isnull().sum().sort_values(ascending=False)

In [ ]:
# selecting only columns with missing values
column_missing_values = azdias.isnull().sum().sort_values(ascending=False)
column_missing_values = column_missing_values[column_missing_values > 0]

print(column_missing_values)
print("There are {} columns with missing values.".format(len(column_missing_values)))

In [ ]:
# Create a histogram of columns with number of missing values
column_missing_values.plot.bar(figsize=(15,10))
plt.title('Columns and number of missing values')
plt.xlabel('Column')
plt.ylabel('Number of missing values')
plt.show()

## Part 1: Customer Segmentation Report

The main bulk of your analysis will come in this part of the project. Here, you should use unsupervised learning techniques to describe the relationship between the demographics of the company's existing customers and the general population of Germany. By the end of this part, you should be able to describe parts of the general population that are more likely to be part of the mail-order company's main customer base, and which parts of the general population are less so.

## Part 2: Supervised Learning Model

Now that you've found which parts of the population are more likely to be customers of the mail-order company, it's time to build a prediction model. Each of the rows in the "MAILOUT" data files represents an individual that was targeted for a mailout campaign. Ideally, we should be able to use the demographic information from each individual to decide whether or not it will be worth it to include that person in the campaign.

The "MAILOUT" data has been split into two approximately equal parts, each with almost 43 000 data rows. In this part, you can verify your model with the "TRAIN" partition, which includes a column, "RESPONSE", that states whether or not a person became a customer of the company following the campaign. In the next part, you'll need to create predictions on the "TEST" partition, where the "RESPONSE" column has been withheld.

In [ ]:
mailout_train = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TRAIN.csv', sep=';')